In [1]:
import pandas as pd
import sqlite3
from time import time, sleep
import sys 
from time import sleep
import numpy as np
from helper_functions import *

In [2]:
df = pd.read_csv("../data/AllNYSVoters.txt", nrows= 10, header=None)
print(df.columns)

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44],
           dtype='int64')


In [3]:
# rename columns
cols = ["Last_Name", "First_Name", "Middle_Name", "Name_Suffix","House_Number", \
       "Half_Code","Apartment","Pre_Street_Direction","Street_Name_Orig","Post_Street_Direction",\
       "City", "Zip5", "Zip4", "Mailing_Address1", "Mailing_Address2", "Mailing_Address3",\
       "Mailing_Address4", "DOB","Gender","Party","Other_Party","County_Code",\
       "Election_District","Legislative_District","Town","Ward","CD","State_Senate",\
        "State_Assembly","Last_Date_Voted","LastYearVoted","PrevCounty", "PrevAddres",\
       "PrevName","CountyID","Reg_Date","AppSource","IDReq","IDmet","Status","Reason","InactDate",\
       "PurgeDate","StateID", "Voter_Hx"]
df.columns = cols
# Columns to read from the raw data
read_cols = ["City", "Zip5","DOB","Gender","Party","County_Code",\
        "CD","State_Senate",\
        "State_Assembly","Reg_Date","Status","StateID", "Voter_Hx"]
# Columns to be used in the final table. 
use_cols = ["City", "Zip5","DOB","Gender","Party","County_Code",\
       "CD","State_Senate",\
        "State_Assembly","Reg_Date","Status","StateID", "Voter_Hx"]

df.head()

2,SALISBURY,LINDA,L,NaN,322,NaN,NaN,NaN,GARDNERSVILLE RD,NaN,...,19731012,CBOE,N,Y,ACTIVE,NaN,NaN,NaN,NY000000000008502650,2017 General Election;11/8/2016 General Electi...
3,WHITBECK,GRACE,M,NaN,103,NaN,NaN,NaN,DOLEN LN,NaN,...,20050420,CBOE,N,Y,PURGED,MOVED,NaN,20140917.0,NY000000000022546641,2013 General Election;2012 General Election;Ge...
4,URBANO,ROBERT,NaN,NaN,573,NaN,NaN,NaN,ENID RD,NaN,...,19920214,CBOE,N,Y,ACTIVE,NaN,NaN,NaN,NY000000000008502796,2017 General Election;11/8/2016 General Electi...


In [4]:
data_types = {
    'City': str,
    'Zip5': np.float64,
    'DOB': str,
    'Gender': str,
    'County_Code': np.float64,
    'CD': np.float64,
    'State_Senate': np.float64,
    'State_Assembly': np.float64,
    'Reg_Date': str,
    'StateID': str,
    'Voter_Hx': str,
}

### Save select Data

In [5]:
# Midterm election dates
elections = ["11/4/14", "11/2/10"]
print(elections)

['11/4/14', '11/2/10']


In [6]:
%timeit
with sqlite3.connect("../data/govt990.db") as db:
    cursor = db.cursor()
    cursor.execute('''
    DROP TABLE IF EXISTS nys_young_voters;''')
    cursor.execute('''
    CREATE TABLE nys_young_voters(
    City TEXT,
    Zip5 INT,
    DOB TEXT,
    Gender TEXT,
    Party TEXT,
    County_Code INT,
    CD INT,
    State_Senate INT,
    State_Assembly INT,
    Last_Date_Voted TEXT,
    Reg_Date TEXT,
    Status TEXT,
    StateID TEXT,
    Voter_Hx TEXT);
    ''')
    count = 0 # init count
    reader = pd.read_csv("../data/AllNYSVoters.txt", names = cols, usecols = read_cols,
                         chunksize = int(5e4), encoding = "latin1", dtype = data_types, header = None,
                         engine = "c")
    for chunk in reader:
        count += len(chunk)
        # Filter out voters who were > 24 years old in the last election I
        # am going to examine
        last_election = elections[len(elections) - 1]
        chunk["Young"] = young_at_election(chunk["DOB"], election_date = last_election)
        chunk[read_cols][chunk["Young"]].to_sql("nys_young_voters", db, if_exists = "append", index = False,
                                               chunksize = int(len(chunk)/4))
        
        if count % (2 * int(5e5)) == 0:
            sleep(15)
        progress(count, total=17784437)